In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
import re

from bs4 import BeautifulSoup as bs

In [4]:
def preprocessing(article):
    MESSAGE = '지원하지 않는 브라우저로 접근하셨습니다.\nInternet Explorer 10 이상으로 업데이트 해주시거나, 최신 버전의 Chrome에서 정상적으로 이용이 가능합니다.'

    article_text = str(article)
        
    pattern = '\[[^\]]*\]' # [] 내부 모두 제거
    tmp = re.sub(pattern, '', article_text)

    tmp = tmp.replace('\n', '').replace('\t', '').replace('\r', '') # 공백 제거
    pattern = "<br/?>" # <br> 태그 -> 개행으로 변경
    tmp = re.sub(pattern, '\n', tmp)
    
    # 캡션 표시
    pattern = re.compile('(<p style="[^>]*>)([^<]*)(</p>)')
    result = pattern.finditer(tmp)
    for r in result:
        tmp = tmp.replace(r.group(), f"[{r.group(2)}]")
    # 캡션 표시
    pattern = re.compile('(<span class="sub_tit">)([^<]*)(</span>)')
    result = pattern.finditer(tmp)

    for r in result:
        tmp = tmp.replace(r.group(), f"[{r.group(2)}]")
    
    # print('after ###### caption eliminate')
    # print(tmp)
    pattern = "</?p[^>]*>" # <p> or </p> -> 개행으로 변경
    tmp = re.sub(pattern, '\n', tmp)
    
    pattern = "<caption>[^>]+>" # caption 제거
    tmp = re.sub(pattern, '', tmp)

    pattern = "<a.+</a>" # [a] 태그 제거
    tmp = re.sub(pattern, '', tmp)

    pattern = "<img[^>]+>" # img들 모두 제거
    tmp = re.sub(pattern, '\n[IMAGE]\n', tmp)
    content = bs(tmp, 'html.parser') # 다시 parsing
    tmp = re.sub(' {2,}', ' ', content.text)

    pattern = "<[^>]*>" # <> 내부 모두 제거
    text = re.sub(pattern, '', tmp)
    
    pattern = "\([^\)]*\)" # () 내부 모두 제거
    text = re.sub(pattern, '', text)
    
    last_text = ('').join([word for word in text if word.isalpha() or ord(word) < 128])
    text = last_text.replace(MESSAGE, '')
    
    pattern = '[a-zA-Z0-9+-_.]+@[a-zA-Z0-9+-_]+.com'
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9+-_.]+@[a-zA-Z0-9+-_]+.co.kr'
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9+-_.]+@'
    text = re.sub(pattern, '', text)
    
    text = re.sub('\n ', '\n\n', text)
    text = re.sub('-\n', '\n\n', text)
    text = re.sub('\n{2,}', '\n\n', text)
    
    while text[0] == ' ' or text[0] == '\n':
        text = text[1:]
    if text[0] == ']': text = text[1:]
    while text[-1] == ' ' or text[-1] == '\n':
        text = text[:-1]
    text = text.replace('기사내용 요약', '[기사내용 요약]\n')

    return text

In [5]:
URL = "https://news.nate.com/view/20230209n11296"

req = requests.get(URL)
content = bs(req.text, 'html.parser')

In [6]:
try:
    # title
    title = content.find('h3', {'class': 'viewTite'})
    if not title:
        title = content.find('h3', {'class': 'articleSubecjt'})
    print('Title: ' + title.text)
except:
    print('Title Error!\n')

try:
    # category
    nav = content.find('div', {'class': 'snbArea'})
    category = nav.find('li', {'class': 'on'})
    print('Category: ', end='')
    print(category.text if category else 'X')
except:
    print('Category Error!\n')
    
try:
    # press
    _press = content.find('a', {'class': 'medium'})
    if _press:
        press = _press.text
    else:
        press = content.find('dl', {'class': 'articleInfo'})
    print('Press: ' + press)
except:
    print('Press Error!\n')

try:
    # date
    date = content.find('em').text
    print('Date: ' + date[:10])
except:
    print('Date Error!\n')

try:
    # content
    article = content.find('div',{'id': 'articleContetns'})
    print(preprocessing(article))
except:
    print('Content Error!\n')

Title: 이상민 탄핵소추의결서 헌재 제출…"신속한 결정 기대"(종합)


Date: 2023-02-09


與소속 김도읍 소추위원"탄핵당할 만큼 법률 위반인지 국민이 판단할 것"
소추의결서 내용엔 "국정조사 내용 그 이상도 이하도 아닌 것 확인돼"

[IMAGE]
[이상민 장관 탄핵소추의결서 헌재 제출

한상균 기자 = 정성희 국회 법제사법위원회 수석전문위원이 9일 오전 헌법재판소에서 이상민 행정안전부 장관 탄핵 소추 의결서를 제출하고 있다. 2023.2.9 ]

이슬기 기자 = 국회는 이상민 행정안전부 장관 탄핵 소추의결서를 9일 오전 10시 헌법재판소에 제출했다.

탄핵 심판에서 검사 역할을 담당할 김도읍 법제사법위원장은 이날 언론에 문자 메시지를 보내 이같이 밝혔다.

김 위원장은 탄핵 소추의결서를 정성희 법사위 수석전문위원에게 위임해 제출했다.

앞서 더불어민주당 등 야 3당은 이태원 참사 부실 대응을 이유로 발의한 이 장관 탄핵 소추안을 전날 국회 본회의에서 여당인 국민의힘 반대를 뚫고 가결시켰다.

헌재로 넘어간 탄핵 심판에서 국민의힘 소속 김도읍 법제사법위원장은 국회법과 헌법재판소법상 이 장관 탄핵을 주장해야 하는 검사 역할을 맡게 된다.

김 위원장은 이날 오전 국회에서 기자들과 만나 "그간 민주당이 주장했던 내용과 소추의결서도 공개돼 있으니 법리적으로  검토할 필요는 없다"며 정치권 안팎의 예상보다 빠르게 의결서를 제출한 이유를 설명했다.

그는 "행안부 장관의 자리를 비워놓을 수밖에 없어서 국정 공백이 생겼다. 고스란히 나라의 손실이고 국민께 피해가 갈 수 있다"며 "국정 공백기를 최소화해야 한다는 점에서 헌재가 심판 절차를 신속하게 했으면 좋겠다"고 말했다.

그러면서 " 재판관님들의 권한 범위내에 있지만, 집중심리 등 배려 속에 심판이 신속하게 결정이 나길 바란다"고 덧붙였다.

[IMAGE]
[발언하는 김도읍 법사위원장
연합뉴스 자료 사진 ]

전날 가결된 소추의결서의 내용과 관련해선 "어제 상당한 물량의 증거자료와 참고자료를 함께 받았고, 그게 고스란히 헌재로 갈 것"이라며 "국민들도 공개적으로 다 보셨겠지만, 국정조사에서 나왔던 내용 그 